In [1]:
import numpy as np
import os
#import matplotlib.pyplot as plt
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
import pandas as pd

import diff_classifier

import mrmr


In [2]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.49.0


### If running in the East US region, run the cell below

In [3]:
ws = Workspace.get(
    name='nels_east_us',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='eastus')

In [7]:
ws.name

'nels_east_us'

### if running in East US 2 region, run the cell below

In [5]:
ws = Workspace.get(
    name='schimek_mpt_eastus2',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='eastus2')

### If running in West US 2, run the cell below

In [3]:
ws = Workspace.get(
    name='schimek_west_us_2',
    subscription_id='9c49afdd-cccb-44e2-a4eb-5b2f941c774c',
    resource_group='schimek_cloud',
    location='westus2')

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
if ws.name == 'nels_east_us':
    cluster_name = 'schimek-gpu-cluster'
elif ws.name == 'schimek_mpt_eastus2':
    cluster_name = "nlsschim-gpu-cluster"
elif ws.name == 'schimek_west_us_2':
    cluster_name = "schimek-cluster-gpu"
print(cluster_name)

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=1,
                                                           min_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

# can poll for a minimum number of nodes and for a specific timeout. 
# if no min node count is provided it uses the scale settings for the cluster
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


schimek-cluster-gpu
Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-06-01T21:47:38.531000+00:00', 'errors': [{'error': {'code': 'ClusterHasWarnings', 'message': 'The specified VMSize STANDARD_NC6 is being deprecated by targetDate: 07/31/2023 More details can be found at : https://learn.microsoft.com/en-us/azure/virtual-machines/n-series-migration. Please use any of the recommended VMSizes mentioned in the link to recreate the compute.'}}], 'creationTime': '2023-05-23T23:23:38.785077+00:00', 'modifiedTime': '2023-05-23T23:24:16.116257+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSetting

In [5]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

schimek-cluster-gpu AmlCompute Succeeded


In [6]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.7
- pip=21.3.1
- py-xgboost-gpu
- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - tensorflow-gpu==2.0.0
  - keras<=2.3.1
  - matplotlib
  - protobuf==3.20.1
  - stochastic
  - diff_classifier
  - scikit-learn
  - polars
  - mrmr-selection

Overwriting conda_dependencies.yml


In [7]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

# Specify a GPU base image
##keras_env.docker.enabled = True
#keras_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.1-cudnn8-ubuntu20.04'

In [8]:
datastore = Datastore.get(ws, 'workspaceblobstore')
file_dataset = Dataset.File.from_files(path=(datastore, 'UI/2023-07-24_192934_UTC/'))



In [9]:
# Confirm the correct dataset is being used
file_dataset.to_path()

['/features_P17_1h_OGD_3d_40nm_slice_1_striatum_vid_1.csv',
 '/features_P17_2h_OGD_3d_40nm_slice_1_striatum_vid_1.csv',
 '/features_P17_3h_OGD_3d_40nm_slice_1_striatum_vid_1.csv',
 '/features_P17_NT_3d_40nm_slice_1_striatum_vid_1.csv']

In [10]:
dataset = file_dataset.register(workspace = ws, name = 'P17_OGD_3div_STR_features', create_new_version=True)

In [17]:
from azureml.core import ScriptRunConfig

#args = ['--input-data', dataset.as_named_input("trajectories")]#,
#         '--batch-size', 50,
#         '--first-layer-neurons', 300,
#         '--second-layer-neurons', 100,

src = ScriptRunConfig(source_directory='.',
                      script='azure_script_accuracy_plateau.py',
                      #arguments=args,
                      compute_target=compute_target,
                      environment=keras_env)

In [18]:
from azureml.core import Experiment
exp = Experiment(workspace=ws, name='azure_3DIV_str_acc_plateau')

In [20]:
run = exp.submit(src)

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…